# Tutorial: Generating Base Population and Household Data

This tutorial will guide you through the process of generating base
population and household data for a specified region using census data.
We’ll use a `CensusDataLoader` class to handle the data
processing and generation.

## Before Starting

Make sure your `population data` and `household data` are in the prescribed format. Names of the column need to be same as shown in the excerpts.

Lets see a snapshot of the data

`Population Data` is a dictionary containing two pandas DataFrames: '`age_gender`' and '`ethnicity`'. Each DataFrame provides demographic information for different areas and regions.

The `age_gender` DataFrame provides a comprehensive breakdown of population data, categorized by area, gender, and age group. 

#### Columns Description

- `area`: Serves as a unique identifier for each geographical area, represented by a string (e.g., `'BK0101'`, `'SI9593'`). 
- `gender`: Indicates the gender of the population segment, with possible values being `'female'` or `'male'`.
- `age`: Specifies the age group of the population segment, using a string format such as `'20t29'` for ages 20 to 29, and `'U19'` for those under 19 years of age. 
- `count`: Represents the total number of individuals within the specified gender and age group for a given area. 
- `region`: A two-letter code that identifies the broader region encompassing the area (e.g., `'BK'` for Brooklyn, `'SI'` for Staten Island).

##### Example Entry

Here is a sample of the data structure within the `age_gender` DataFrame:

| area  | gender | age  | count | region |
|-------|--------|------|-------|--------|
| BK0101| female | 20t29| 3396  | BK     |
| BK0101| male   | 20t29| 3327  | BK     |

This example entry demonstrates the DataFrame's layout and the type of demographic data it contains, highlighting its utility for detailed population studies by age and gender.

The `ethnicity` DataFrame is structured to provide detailed population data, segmented by both geographical areas and ethnic groups.

##### Columns Description

- `area`: A unique identifier assigned to each area, formatted as a string (e.g., `'BK0101'`, `'SI9593'`). This identifier helps in pinpointing specific locations within the dataset.
- `ethnicity`: Represents the ethnic group of the population in the specified area.
- `count`: Indicates the number of individuals belonging to the specified ethnic group within the area. This is an integer value representing the population count.
- `region`: A two-letter code that signifies the broader region that the area belongs to (e.g., `'BK'` for Brooklyn, `'SI'` for Staten Island). 

##### Example Entry

Below is an example of how the data is presented within the DataFrame:

| area  | ethnicity | count | region |
|-------|-----------|-------|--------|
| BK0101| asian     | 1464  | BK     |
| BK0101| black     | 937   | BK     |

This example illustrates the structure and type of data contained within the `ethnicity` DataFrame, showcasing its potential for detailed demographic studies.

`Household Data` contains the following columns:

- `area`: Represents a unique identifier for each area.
- `people_num`: The total number of people within the area.
- `children_num`: The number of children in the area.
- `household_num`: The total number of households.
- `family_households`: Indicates the number of households identified as family households, highlighting family-based living arrangements.
- `nonfamily_households`: Represents the number of households that do not fall under the family households category, including single occupancy and unrelated individuals living together.
- `average_household_size`: The average number of individuals per household.

Below is a sample excerpt:

| area  | people_num | children_num | household_num | family_households | nonfamily_households | average_household_size |
|-------|------------|--------------|---------------|-------------------|----------------------|------------------------|
| 100100| 104        | 56           | 418           | 1                 | 0                    | 2.488038               |
| 100200| 132        | 73           | 549           | 1                 | 0                    | 2.404372               |
| 100300| 5          | 0            | 10            | 0                 | 1                    | 5.000000               |




Now that we have verified our input, we can proceed to next steps!

## Step 1: Set Up File Paths

First, we need to specify the paths to our data files.

Make sure to replace the placeholder paths with the actual paths to your
data files.

In [1]:
# Path to the population data file. Update with the actual file path.
# import os
# os.chdir("")
dir_name = "docs/tutorials/processing-a-population/"
POPULATION_DATA_PATH = "sample_data/NYC/population.pkl"

# Path to the household data file. Update with the actual file path.
HOUSEHOLD_DATA_PATH = "sample_data/NYC/household.pkl"

## Step 2: Define Age Group Mapping

We’ll define a mapping for age groups to categorize adults and children
in the household data:


In [2]:
AGE_GROUP_MAPPING = {
    "adult_list": ["20t29", "30t39", "40t49", "50t64", "65A"],  # Age ranges for adults
    "children_list": ["U19"],  # Age range for children
}

## Step 3: Load Data

Now, let’s load the population and household data:


In [3]:
import numpy as np
import pandas as pd

# Load household data
HOUSEHOLD_DATA = pd.read_pickle(HOUSEHOLD_DATA_PATH)

# Load population data
BASE_POPULATION_DATA = pd.read_pickle(POPULATION_DATA_PATH)

In [4]:
BASE_POPULATION_DATA['ethnicity']

,area,ethnicity,count,region
0,BK0101,asian,1464,BK
1,BK0101,black,937,BK
2,BK0101,hispanic,5498,BK
3,BK0101,native,22,BK
4,BK0101,other,2324,BK
...,...,...,...,...
1567,SI9593,black,0,SI
1568,SI9593,hispanic,0,SI
1569,SI9593,native,0,SI
1570,SI9593,other,0,SI


## Step 4: Set Up Additional Parameters

We’ll set up some additional parameters that might be needed for data
processing. These are not essential for generating population, but still good to know if you decide to use them in future.

In [5]:
# Placeholder for area selection criteria, if any. Update or use as needed.
# Example: area_selector = ["area1", "area2"]
# This will be used to filter the population data to only include the selected areas.
area_selector = None

# Placeholder for geographic mapping data, if any. Update or use as needed.
geo_mapping = None

## Step 5: Initialize the Census Data Loader

Create an instance of the `CensusDataLoader` class:

In [6]:
from agent_torch.data.census.census_loader import CensusDataLoader

census_data_loader = CensusDataLoader(n_cpu=8, use_parallel=True)

/home/pcarragh/miniconda3/envs/agent_torch_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-04 17:19:32,056	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


This initializes the loader with 8 CPUs and enables parallel processing
for faster data generation.

## Step 6: Generate Base Population Data

Generate the base population data for a specified region:

In [7]:
census_data_loader.generate_basepop(
    input_data=BASE_POPULATION_DATA,  # The population data frame
    region="astoria",  # The target region for generating base population
    area_selector=area_selector,  # Area selection criteria, if applicable
)

2025-09-04 17:19:33,599	INFO worker.py:1951 -- Started a local Ray instance.
INFO:agent_torch.data.census.generate.base_pop:Processing: 0/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 1/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 2/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 3/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 4/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 5/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 6/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 7/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 8/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 9/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 10/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 11/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 12/262
INFO:agent_torch.data.census.generate.base_pop:Processing: 13/262
INFO:agen

This will create a base population of 100 individuals for the “astoria”
region. The generated data will be exported to a folder named “astoria”
under the “populations” folder.

#### Overview of the Generated Base Population Data

Each row corresponds to attributes of individual residing in the specified region while generating the population.

| area  | age  | gender | ethnicity | region |
|-------|------|--------|-----------|--------|
| BK0101| 20t29| female | black     | BK     |
| BK0101| 20t29| female | hispanic  | BK     |
| ...   | ...  | ...    | ...       | ...    |
| BK0101| U19  | male   | asian     | SI     |
| BK0101| U19  | female | white     | SI     |
| BK0101| U19  | male   | asian     | SI     |


## Step 7: Generate Household Data

Finally, generate the household data for the specified region:


In [8]:
census_data_loader.generate_household(
    household_data=HOUSEHOLD_DATA,  # The loaded household data
    household_mapping=AGE_GROUP_MAPPING,  # Mapping of age groups for household composition
    region="astoria"  # The target region for generating households
)

2025-09-04 17:20:01,439	INFO worker.py:1951 -- Started a local Ray instance.
INFO:agent_torch.data.census.generate.household:0/214: Processing BK0101
/home/pcarragh/dev/wip/AgentTorchIE/agent_torch/data/census/generate/household.py:447: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_household_data["adult_num"] = (
/home/pcarragh/dev/wip/AgentTorchIE/agent_torch/data/census/generate/household.py:451: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_household_data["adult_num"] = proc_household_data["adu

(create_household_composition_v3_remote pid=754861) Starting create_household_composition_v3...


(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=754865) 
(create_household_composition_v3_remote pid=75

(create_household_composition_v3_remote pid=754863) Starting create_household_composition_v3... [repeated 137x across cluster]


(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
(create_household_composition_v3_remote pid=754863) 
2025-09-04 17:20:09,806	ERROR worker.py:429 --

RayTaskError(KeyError): [36mray::create_household_composition_v3_remote()[39m (pid=754861, ip=128.2.205.151)
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'living_alone'

The above exception was the direct cause of the following exception:

[36mray::create_household_composition_v3_remote()[39m (pid=754861, ip=128.2.205.151)
  File "/home/pcarragh/dev/wip/AgentTorchIE/agent_torch/data/census/generate/household.py", line 175, in create_household_composition_v3_remote
    return create_household_composition_v3(
  File "/home/pcarragh/dev/wip/AgentTorchIE/agent_torch/data/census/generate/household.py", line 216, in create_househol

This will create household data for the “astoria” region based on the
previously generated base population. The generated data will be
exported to the same “astoria” folder under the “populations” folder.

## Bonus: Generate Population Data of Specific Size

For quick experimentation, this may come in handy.

In [ ]:
census_data_loader.generate_basepop(
    input_data=BASE_POPULATION_DATA,  # The population data frame
    region="astoria",  # The target region for generating base population
    area_selector=area_selector,  # Area selection criteria, if applicable
    num_individuals = 100 # Saves data for first 100 individuals, from the generated population
)

## Bonus: Export Population Data

If you have already generated your synthetic population, you just need to export it to "populations" folder under the desired "region", in order for you to use it with AgentTorch.

In [ ]:
POPULATION_DATA_PATH = "/population_data.pickle"  # Replace with actual path
census_data_loader.export(population_data_path=POPULATION_DATA_PATH,region="astoria")

In case you want to export data for only few individuals

In [ ]:
census_data_loader.export(population_data_path=POPULATION_DATA_PATH,region="astoria",num_individuals = 100)

## Conclusion

You have now successfully generated both base population and household
data for the `“astoria”` region. The generated data can be found in the
`“populations/astoria”` folder. You can modify the region name, population
size, and other parameters to generate data for different scenarios.
